In [ ]:
import pandas as pd



# pd.set_option('display.max_colwidth', None)
pd.reset_option('display.max_colwidth')

data1 = pd.read_csv('../data/appartements1.csv')
data2 = pd.read_csv('../data/appartements2.csv')
appartements=pd.concat([data1,data2],axis=0,ignore_index=True)
df =appartements.copy()



In [ ]:

##-pre-processing and preparation of the data 
from sklearn.model_selection import train_test_split
from preprocessing import Preprocessing 
p =Preprocessing()
X,y,vectorize =p.processing(df)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2,random_state=0)


c:\Users\us\Desktop\housePricePrediciton-FInal\processing\preprocessing.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['etat'] =df['etat'].str.lower().str.replace(' ','')
c:\Users\us\Desktop\housePricePrediciton-FInal\processing\preprocessing.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['autre'] =df['autre'].str.lower()
c:\Users\us\Desktop\housePricePrediciton-FInal\processing\preprocessing.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

### Optimisation 

In [ ]:
import pickle

file =open('../Models/Stacking.pkl','rb')
Stacking =pickle.load(file)



In [ ]:
## Reformation of the dataframe 

df=pd.concat([X_train,X_test],axis=0)
y=pd.concat([y_train,y_test],axis=0)

df =pd.concat([df,y],axis=1)


In [ ]:
import pandas as pd
data_dict ={'surface(m²)':120,
'pieces':3,
'chambres':2,
'salles de bains':3,
'etat':'bonétat',
'etage':'3etage',
'ans':'10to20',
'ville':'casablanca',
'autre' :'parquet tv'
}
prediction =pd.DataFrame([data_dict])

In [ ]:
prediction

,surface(m²),pieces,chambres,salles de bains,etat,etage,ans,ville,autre
0,120,3,2,3,bonétat,3etage,10to20,casablanca,parquet tv


In [ ]:
import numpy as np

## this function prepare the value to be predicted 
def prepare_data_for_prediction(df):
    numerica_cols =df.select_dtypes(include=np.number)

    df['Description'] = (df['etat'].fillna('') +" " +df['etage'].fillna('') +" "+
                    " "+ df['ans'].fillna('') +" "+ df['ville'].fillna('') +" "+
                        df['autre'].fillna('')).str.replace('  ',' ').str.replace('nan','').str.lower().str.strip()
    v=df['Description']
    pd.set_option('display.max_columns', None)
    
    ##--apply TF-IDF on the appartment we want to predict the price 
    X =vectorize.transform(v)
    X_tfidf_pred =pd.DataFrame(X.toarray(),columns=vectorize.get_feature_names_out())
    numeric_cols_pred =prediction[list(numerica_cols)]
    to_predict =pd.concat([numeric_cols_pred,X_tfidf_pred],axis=1)

    return to_predict


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity,pairwise_distances
###---calculate the prediction 

to_predict =prepare_data_for_prediction(prediction)
price =Stacking.predict(to_predict)
#--- Add predicted price 
to_predict['prix(DHs)']=price



print(f"the price of this apartment will be almost: {price[0]} DHs ")

the price of this apartment will be almost: 1120621.7101436039 DHs 


In [ ]:

to_predict.shape

(1, 247)

In [ ]:
##-------calculate the similarity by cosine metric
similarities = cosine_similarity(df, to_predict)
df['similarity'] = similarities

##-----get the index of the the most similar appartment to which we predicted the price 
index =df[df['similarity']==df['similarity'].max()].index[0]

##--find the origine record in  the appartements dataFrame
similare_att =appartements[appartements.index==index]
print(f"you can visit recommanded appartment from this link :\n{similare_att['lienArticle'].values[0]}")

you can visit recommanded appartment from this link :
https://www.mubawab.ma/fr/a/5980725/bel-appartement-a-vendre-au-centre-de-mohammedia


In [ ]:
similare_att

,surface(m²),pieces,chambres,salles de bains,etat,etage,ans,ville,autre,prix(DHs),lienImage,lienArticle
7263,131 m²,3 Pièces,2 Chambres,2 Salles de bains,Nouveau,1er\n\t\t\t\t\tétage,5-10 ans,Quartier du Parc à Mohammedia,Terrasse Garage Ascenseur Concierge Chambre ra...,1650000 DH,https://www.mubawab-media.com/ad/5/980/725F/h/...,https://www.mubawab.ma/fr/a/5980725/bel-appart...


In [ ]:
df['similarity'].max()

0.813857247126144

In [ ]:
df

,surface(m²),pieces,chambres,salles de bains,10etage,10to20,11etage,12etage,13etage,14etage,15etage,16etage,17etage,18etage,19etage,1etage,1to5,20etage,20to30,2etage,30to50,3etage,4etage,50to70,5etage,5to10,6etage,70to100,7etage,8etage,9etage,abadou,adrej,aferkat,agadir,agadirmelloul,ainaouda,ainattig,aincheggag,ainchouater,ainnzagh,aitbaha,aitbazza,aitmelloul,aitourir,aittaguella,alhoceima,aloued,anjra,antenne,aourir,aousserd,ascenseur,asilah,azemmour,azla,azrou,aïnharrouda,benguerir,benmansour,benslimane,berkane,berrechid,birjdid,blindée,bnilent,bnisaid,bonétat,boukhalef,bouskoura,bouznika,bénimellal,béniyakhlef,cabonegro,carrelage,casablanca,central,chambre,chauffage,chefchaouen,chellalate,cheminée,cherrat,chrafat,climatisation,concierge,cuisine,dakhla,darbnikarrich,darbouazza,dcheïraeljihadia,deroua,diq,double,drargua,elgara,elhajeb,eljadida,elkelâadessraghna,elmajjatiaouledtaleb,elmansouria,elmenzeh,errachidia,errahma,essaouira,est,européen,extérieure,façade,fnideq,four,fquihbensaleh,fès,garage,gmassa,goulmima,guelmim,guercif,gueznaia,hadsoualem,harbil,harhoura,ifrane,imouzzer,imzouren,inzegan,jardin,kasbatadla,kendar,khouribga,khémisset,ksarbjir,ksarel,ksarsghir,kébir,kénitra,laaouama,lahraouyine,lakhiaita,lamharzaessahel,larache,laver,laäyoune,les,lqliaa,machine,marbre,marocain,marrakech,martil,matmata,mehdia,meknassaalgharbia,meknes,mer,merselkheir,meublé,micro,midelt,mirleft,mohammedia,montagnes,morethan100,moulayabdallah,moulayyaâcoub,méchouarfèsjdid,médiouna,nador,nord,nouaceur,nouveau,ondes,oualidia,ouarzazate,ouazzane,ouedlaou,ouedzem,ouest,ouislane,oujda,oukaimden,ouladsalah,ouladtayeb,ouladzidane,ourika,parabolique,parquet,piscine,porte,rabat,rangement,raselma,rissanajanoubia,réfrigérateur,saada,saadla,safi,salon,salé,saïdia,sefrou,selouane,settat,sick,sidiabdallahghiat,sidiallalelbahraoui,sidibennour,sidibibi,sidibouknadel,sidibouzid,sidighanem,sidikacem,sidirahal,sidirahalchatai,sidislimane,sidislimaneechcharraa,sidiyahyazaer,skhirat,souksebtoulednemma,sud,sur,sécurité,taghazout,tamesna,tamri,tanger,tanougha,taourirt,taroudant,tassoultante,taza,temara,terrasse,tighassaline,titmellil,tiznit,tnineaglou,tv,tétouan,vitrage,vue,youssoufia,zaouiatsidikacem,zenata,àrénover,équipée,prix(DHs),similarity
11299,106.0,300.000000,200.0,200.000000,0.0,310.177554,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0000,0.0,0.00000,570.59853,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,158.917591,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,239.424341,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,241.032387,194.408031,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,228.160300,228.160300,0.0,0.000000,0.0,0.0,189.453922,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,278.532176,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,382.221882,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,189.885543,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,1650.0,0.702019
4221,70.0,300.000000,200.0,213.289698,0.0,250.588567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,292.845833,0.0000,0.0,0.00000,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,150.286934,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,128.387534,0.0,0.0,0.0,0.0,0.0,0.0,196.676105,193.427931,0.0,0.000000,0.0,0.0,0.0,0

In [ ]:
((df==0).sum()/df.shape[0]).sort_values(ascending=False)[225:245]

four           0.671759
façade         0.643696
extérieure     0.643696
marocain       0.640962
parabolique    0.630749
antenne        0.630749
sécurité       0.586201
salon          0.550901
1etage         0.520907
concierge      0.518736
terrasse       0.498955
garage         0.497025
ascenseur      0.481425
équipée        0.457945
cuisine        0.457945
5to10          0.390720
bonétat        0.339820
prix(DHs)      0.000000
surface(m²)    0.000000
pieces         0.000000
dtype: float64